# devlog 2024-08-13

Testing the `@adrio_cache` method for adding cache behavior to ADRIOs.

In [1]:
import numpy as np
from numpy.typing import NDArray

from epymorph import *
from epymorph.adrio.adrio import Adrio, adrio_cache
from epymorph.geography.us_census import StateScope
from epymorph.simulator.data import evaluate_param

## Test 1:

A non-caching ADRIO will be evaluated every time.

In [2]:
class Cosine(Adrio[np.float64]):
    """Trivial ADRIO -- calculate a cosine curve."""

    requirements = [AttributeDef("gamma", float, Shapes.S)]

    def evaluate(self) -> NDArray[np.float64]:
        print("!!! calculating cosine !!!")
        T = self.dim.days
        t = np.arange(0, T)
        gamma = self.data("gamma")
        return gamma * np.cos(2 * np.pi * t / T)


rume1 = SingleStrataRume.build(
    ipm_library["sirs"](),
    mm_library["no"](),
    init.NoInfection(),
    StateScope.in_states(["04"]),
    TimeFrame.of("2020-01-01", 20),
    {
        "population": 100,
        "beta": Cosine(),
        "gamma": 2.0,
        "xi": 1 / 10,
    },
)

print("We should see '!!! calculating cosine !!!' three times...")
sim = BasicSimulator(rume1)
out = sim.run()
out = sim.run()
out = sim.run()

We should see '!!! calculating cosine !!!' three times...
!!! calculating cosine !!!
!!! calculating cosine !!!
!!! calculating cosine !!!


## Test 2:

A caching ADRIO will be re-evaluated if its requirements change.

In [3]:
@adrio_cache
class CachedCosine(Adrio[np.float64]):
    """Trivial ADRIO -- calculate a cosine curve."""

    requirements = [AttributeDef("gamma", float, Shapes.S)]

    def evaluate(self) -> NDArray[np.float64]:
        print("!!! calculating cosine !!!")
        T = self.dim.days
        t = np.arange(0, T)
        gamma = self.data("gamma")
        return gamma * np.cos(2 * np.pi * t / T)


rume2 = SingleStrataRume.build(
    ipm_library["sirs"](),
    mm_library["no"](),
    init.NoInfection(),
    StateScope.in_states(["04"]),
    TimeFrame.of("2020-01-01", 20),
    {
        "population": 100,
        "beta": CachedCosine(),
        "gamma": 2.0,
        "xi": 1 / 10,
    },
)

print("We should see '!!! calculating cosine !!!' only twice...")
sim = BasicSimulator(rume2)
out = sim.run()
out = sim.run()
out = sim.run()
out = sim.run({"gamma": 22.0})

We should see '!!! calculating cosine !!!' only twice...
!!! calculating cosine !!!
!!! calculating cosine !!!


## Test 3:

If the scope or any SimDimension changes, the value should recalculate. (This is kind of an odd use-case, to share ADRIOs between RUMEs, but I wouldn't want this to cause problems if the user did it.)

In [4]:
@adrio_cache
class CachedCosine2(Adrio[np.float64]):
    """Trivial ADRIO -- calculate a cosine curve."""

    def evaluate(self) -> NDArray[np.float64]:
        print("!!! calculating cosine !!!")
        T = self.dim.days
        t = np.arange(0, T)
        return 2.0 * np.cos(2 * np.pi * t / T)


cosine = CachedCosine2()

# Base RUME
rume3 = SingleStrataRume.build(
    ipm_library["sirs"](),
    mm_library["no"](),
    init.NoInfection(),
    StateScope.in_states(["04"]),
    TimeFrame.of("2020-01-01", 20),
    {
        "population": 100,
        "beta": cosine,
        "gamma": 2.0,
        "xi": 1 / 10,
    },
)

# Nothing changed...
rume3b = SingleStrataRume.build(
    ipm_library["sirs"](),
    mm_library["no"](),
    init.NoInfection(),
    StateScope.in_states(["04"]),
    TimeFrame.of("2020-01-01", 20),
    {
        "population": 100,
        "beta": cosine,
        "gamma": 2.0,
        "xi": 1 / 10,
    },
)

# Change scope
rume4 = SingleStrataRume.build(
    ipm_library["sirs"](),
    mm_library["no"](),
    init.NoInfection(),
    StateScope.in_states(["35"]),
    TimeFrame.of("2020-01-01", 20),
    {
        "population": 100,
        "beta": cosine,
        "gamma": 2.0,
        "xi": 1 / 10,
    },
)

# Change start date
rume5 = SingleStrataRume.build(
    ipm_library["sirs"](),
    mm_library["no"](),
    init.NoInfection(),
    StateScope.in_states(["04"]),
    TimeFrame.of("2019-01-01", 20),
    {
        "population": 100,
        "beta": cosine,
        "gamma": 2.0,
        "xi": 1 / 10,
    },
)

# Change duration
rume6 = SingleStrataRume.build(
    ipm_library["sirs"](),
    mm_library["no"](),
    init.NoInfection(),
    StateScope.in_states(["04"]),
    TimeFrame.of("2020-01-01", 99),
    {
        "population": 100,
        "beta": cosine,
        "gamma": 2.0,
        "xi": 1 / 10,
    },
)

print("should only see '!!! calculating cosine !!!' once here...")
res1 = evaluate_param(rume3, "beta")
res1 = evaluate_param(rume3, "beta")
res1 = evaluate_param(rume3, "beta")
res1 = evaluate_param(rume3b, "beta")
print("\nand then we should see it three more times...")
res1 = evaluate_param(rume4, "beta")
res1 = evaluate_param(rume5, "beta")
res1 = evaluate_param(rume6, "beta")

should only see '!!! calculating cosine !!!' once here...
!!! calculating cosine !!!

and then we should see it three more times...
!!! calculating cosine !!!
!!! calculating cosine !!!
!!! calculating cosine !!!
